In [ ]:
import sys

import open3d as o3d
import torch

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.data.rv_scan import RVScan
from skrgbd.calibration.calibrations.small_scale_sphere import Calibration

In [ ]:
calibration = Calibration()

## Check in 3D

In [ ]:
for scan_i in range(24):
    scan = RVScan(
        '/home/universal/Downloads/dev.sk_robot_rgbd_data/stl_shared_folder/scans/test_calib_board_folder', scan_i)
    
    mm_to_meters = torch.zeros(4, 4)
    mm_to_meters[3, 3] = 1
    mm_to_meters[0, 0] = mm_to_meters[1, 1] = mm_to_meters[2, 2] = 1 / 1000

    matrix = (
        calibration.stl_sphere_extrinsics[scan_i] @ calibration.rv_calib_to_stl_right
        @ mm_to_meters @ scan.board_to_mesh.inverse() @ scan.mesh_to_world.inverse()
    )
    scan.vertices = scan.vertices @ matrix[:3, :3].T + matrix[:3, 3]
    
    o3d.io.write_triangle_mesh(f'/tmp/{scan_i}.ply', scan.mesh)

## Check in 2D

In [ ]:
from PIL import Image

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/debug/depth_map_reprojection_example/dev.mvs4df/src')
from mvs4df.modules.pointcloud_rendering.render_points import render_points

In [ ]:
camera_model = calibration.cam_model['stl_right']
w, h = camera_model.size_wh

device = 'cuda:0'
camera_model = camera_model.to(device)

In [ ]:
for scan_i in range(24):
    scan = RVScan(
        '/home/universal/Downloads/dev.sk_robot_rgbd_data/stl_shared_folder/scans/test_calib_board_folder', scan_i)
    scan.load_colors()
    
    mm_to_meters = torch.zeros(4, 4)
    mm_to_meters[3, 3] = 1
    mm_to_meters[0, 0] = mm_to_meters[1, 1] = mm_to_meters[2, 2] = 1 / 1000

    matrix = (
        calibration.stl_sphere_extrinsics[3].inverse() @ calibration.stl_sphere_extrinsics[scan_i]
        @ calibration.rv_calib_to_stl_right @ mm_to_meters
        @ scan.board_to_mesh.inverse() @ scan.mesh_to_world.inverse()
    )
    vertices = scan.vertices @ matrix[:3, :3].T + matrix[:3, 3]
    vertices = vertices.T
    
    uv = camera_model.project_fine(vertices.to(device)).cpu()
    torch.cuda.empty_cache()

    render = render_points(
        scan.colors.unsqueeze(0),
        vertices[2].unsqueeze(0),
        uv.unsqueeze(1),
        (h, w),
        point_radius=(2 ** -.5),
    #     uv_averaging_range=1e-4,
        depth_averaging_range=1e-5,
    ).squeeze(0)
    
    render = render.where(render.isfinite(), render.new_zeros([]))
    render = render.permute(1, 2, 0).clamp(0, 255).byte()
    Image.fromarray(render.numpy()).save(f'/tmp/{scan_i}.png')